In [58]:
import joblib
import pandas as pd
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler

In [59]:
root = ".."
input_path = f"{root}/data/augmented"
output_path = f"{root}/data/processed"
input_tables = ["spraying_performance_x5", "droplet_deposition_x10"]

datasets = [pd.read_csv(f"{input_path}/{name}.csv") for name in input_tables]

In [60]:
def expand_collapsed_columns(df, columns_to_expand):
    df = df.copy()
    for col in columns_to_expand:
        if col in df.columns:
            df[f"{col}.min"] = df[col]
            df[f"{col}.max"] = df[col]
            df.drop(columns=[col], inplace=True)
    return df

def unified_dataset(tables, expand_columns):
    expanded_tables = [expand_collapsed_columns(df, expand_columns) for df in tables]
    return pd.concat(expanded_tables, ignore_index=True, sort=False)

In [61]:
# Преобразование столбцов таких как "experiment.weather.temperature" в "(--//--).min" и "(--//--).max"
collapsed_columns = [
    "experiment.weather.temperature",
    "experiment.weather.humidity",
    "experiment.weather.velocity",
]
merged = unified_dataset(datasets, collapsed_columns)

# Объединение таблиц с дополнением (None) если столбцы/значения отсутствуют
merged.to_csv(f"{output_path}/merged.csv", index=False)

In [62]:
# Анализ столбцов с нулевыми значениями

nulls = merged.isnull().sum().reset_index()
nulls.columns = ['column', 'null_count']
nulls_nonzero = nulls[nulls['null_count'] != 0]
nulls_sorted = nulls_nonzero.sort_values(by='null_count', ascending=False)
nulls_sorted

,column,null_count
29,experiment.params.atomization_diameter,140
36,plant.phenotypes.height,120
37,model.positioning_mode,120
38,experiment.params.flow_rate,120
1,plant.cultivar,90
3,model.weight,90
8,model.rotors.power,90
12,model.particle_diameter.min,90
13,model.particle_diameter.max,90
17,model.water_pump.flow_rate,90


In [63]:
def split_targets(df):
    base_features = [col for col in df.columns if not col.startswith('experiment.results.')]
    
    df_cov = df[base_features + ['experiment.results.coverage.value']].copy()
    df_cov = df_cov.dropna(subset=['experiment.results.coverage.value'])

    df_drop = df[base_features + ['experiment.results.droplet_size.value']].copy()
    df_drop = df_drop.dropna(subset=['experiment.results.droplet_size.value'])

    return df_cov, df_drop

In [64]:
# Заменим отсутствующие значения в atomization_diameter на "off"
merged["experiment.params.atomization_diameter"] = merged["experiment.params.atomization_diameter"].fillna("off")

In [65]:
# Категориальные признаки, в которых NaN — отсутствие свойства. Для CatBoost оставляем как есть
catboost_full = merged.copy()
catboost_coverage, catboost_droplet_size = split_targets(catboost_full)
catboost_full.to_csv(f"{output_path}/catboost/full.csv", index=False)
catboost_coverage.to_csv(f"{output_path}/catboost/coverage.csv", index=False)
catboost_droplet_size.to_csv(f"{output_path}/catboost/droplet_size.csv", index=False)

print("CatBoost shapes:")
print(f"Full: {catboost_full.shape}")
print(f"Coverage: {catboost_coverage.shape}")
print(f"Droplet size: {catboost_droplet_size.shape}")

CatBoost shapes:
Full: (210, 39)
Coverage: (210, 34)
Droplet size: (150, 34)


In [66]:
# Для остальных моделей — удалим все столбцы, содержащие хотя бы один NaN (кроме experiment.results.*)
columns_to_keep = [col for col in merged.columns if col.startswith("experiment.results.")]
non_result_cols = [col for col in merged.columns if not col.startswith("experiment.results.") and merged[col].isnull().sum() == 0]
not_null_data = merged[columns_to_keep + non_result_cols]

In [67]:
# Кодируем категориальные признаки и сохраняем отображение

# Определим категориальные признаки
category_cols = not_null_data.select_dtypes(include=['object']).columns.tolist()

# Создание и обучение OneHotEncoder
ohe = OneHotEncoder(sparse_output=False, handle_unknown='ignore')
encoded_array = ohe.fit_transform(not_null_data[category_cols])
encoded_feature_names = ohe.get_feature_names_out(category_cols)

# Создание DataFrame с закодированными признаками
encoded_data = pd.DataFrame(encoded_array, columns=encoded_feature_names, index=not_null_data.index)

# Объединение с остальными признаками
numerical_data = not_null_data.drop(columns=category_cols)
encoded_full = pd.concat([numerical_data, encoded_data], axis=1)
encoded_coverage, encoded_droplet_size = split_targets(encoded_full)

# Сохранение закодированного датафрейма
encoded_full.to_csv(f"{output_path}/encoded/full.csv", index=False)
encoded_coverage.to_csv(f"{output_path}/encoded/coverage.csv", index=False)
encoded_droplet_size.to_csv(f"{output_path}/encoded/droplet_size.csv", index=False)

# Сохраняем энкодер для обратного преобразования
joblib.dump(ohe, f"{output_path}/encoded/ohe_encoder.joblib")

print("XGBoost/GBR/FR shapes:")
print(f"Full: {encoded_full.shape}")
print(f"Coverage: {encoded_coverage.shape}")
print(f"Droplet size: {encoded_droplet_size.shape}")

XGBoost/GBR/FR shapes:
Full: (210, 71)
Coverage: (210, 66)
Droplet size: (150, 66)


In [68]:
# Масштабируем числовые признаки и сохраняем отображение

# Найдём бинарные признаки (0 и 1), в частности категориальные
binary_cols = [col for col in encoded_full.columns
               if encoded_full[col].dropna().nunique() <= 2 and
               set(encoded_full[col].dropna().unique()).issubset({0, 1})]

# Остальные — непрерывные числовые
num_cols = [col for col in encoded_full.columns if col not in binary_cols]

# Масштабируем числовые признаки
scaler = StandardScaler()
scaled_data = scaler.fit_transform(encoded_full[num_cols])

# Объединяем обратно с категориальными признаками
binary_data = encoded_full[binary_cols]
scaled_numeric = pd.DataFrame(scaled_data, columns=num_cols, index=encoded_full.index)
scaled_full = pd.concat([scaled_numeric, binary_data], axis=1)
scaled_coverage, scaled_droplet_size = split_targets(scaled_full)

# Сохранение масштабированного датафрейма
scaled_full.to_csv(f"{output_path}/scaled/full.csv", index=False)
scaled_coverage.to_csv(f"{output_path}/scaled/coverage.csv", index=False)
scaled_droplet_size.to_csv(f"{output_path}/scaled/droplet_size.csv", index=False)

# Сохраняем scaler для будущего восстановления
joblib.dump(scaler, f"{output_path}/scaled/std_scaler.joblib")

print("SRV/MLR shapes:")
print(f"Full: {scaled_full.shape}")
print(f"Coverage: {scaled_coverage.shape}")
print(f"Droplet size: {scaled_droplet_size.shape}")

SRV/MLR shapes:
Full: (210, 71)
Coverage: (210, 66)
Droplet size: (150, 66)
